In [1]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
# from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
# import numpy as np
# import sklearn
import pickle
from keras.applications import *
from keras.preprocessing.image import *
from keras.losses import mse, binary_crossentropy
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import random,math
from tqdm import tqdm ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# mean = np.load('feature/fbank/mean.npy')
# std = np.load('feature/fbank/std.npy')
autoencoder = load_model('model/lgd_dense_AE3.h5')
X_un_dense = np.load('feature/fbank2/semi/X_un_dense.npy')
Y_un_dense = np.load('feature/fbank2/semi/Y_un_dense.npy')
X_test_dense = np.load('feature/fbank2/semi/X_test_dense.npy')
Y_test_dense = np.load('feature/fbank2/semi/Y_test_dense.npy')

base_path = 'feature/fbank2/'#'/tmp2/b03902110/newphase1'
base_data_path = 'feature/fbank2/'#os.path.join(base_path, 'data')
num_fold = 10

In [3]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

def getTrainData():
    X = []
    y = []

    for i in range(num_fold):
        fileX = os.path.join(base_data_path, 'X/X' + str(i+1) + '.npy')
        fileY = os.path.join(base_data_path, 'y/y' + str(i+1) + '.npy')
        
        X.append(np.load(fileX))
        y.append(np.load(fileY))

    X = np.array(X)
    y = np.array(y)

    return X, y

def split_data(X, y, idx):
    X_train = []
    y_train = []
    
    for i in range(num_fold):
        if i == idx:
            X_val = X[i]
            y_val = y[i]
            continue
        if X_train == []:
            X_train = X[i]
            y_train = y[i]
        else:
            X_train = np.concatenate((X_train, X[i]))
            y_train = np.concatenate((y_train, y[i]))

    return X_train, y_train, X_val, y_val

# def normalize(X_train, X_val):
#     X_train = (X_train - mean)/std
#     X_val = (X_val - mean)/std

#     return X_train, X_val


In [4]:
val_set_num = [x+1 for x in range(num_fold)]#str(sys.argv[1])
X, y = getTrainData()
X_train, Y_train, X_val, Y_valid = split_data(X, y, val_set_num[0])
# X_train, X_val = normalize(X_train, X_val)

model = Model(input = autoencoder.layers[0].input, output = autoencoder.layers[5].output)
model.summary()
X_train = model.predict(X_train)
X_valid = model.predict(X_val)
print(X_train.shape,X_valid.shape,Y_train.shape,Y_valid.shape)
X_semi = np.concatenate((X_un_dense,X_test_dense))
Y_semi = np.concatenate((Y_un_dense,Y_test_dense))
X_semi , Y_semi = _shuffle(X_semi,Y_semi)
Y_semi = to_categorical(Y_semi,num_classes=41)
print(X_semi.shape,Y_semi.shape)


/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 63, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 63, 64)       1856      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 63, 32)       57376     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 63, 1)        897       
_________________________________________________________________
flatten_1 (Flatten)          (None, 8064)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 8064)              65036160  
Total params: 65,096,289
Trainable params: 65,096,289
Non-trainable params: 0
________________________________________________________________

In [5]:
X = np.concatenate((X_train,X_semi))
X = np.concatenate((X,X_valid))
mean = X.mean()
std = X.std()
print(mean,std)

24.47078 73.536835


In [6]:
X_train = (X_train - mean) / std
X_valid = (X_valid - mean) / std
X_semi = (X_semi - mean) / std
print(X_train.shape , X_semi.shape , X_valid.shape , X_un_dense.shape , X_test_dense.shape)

(3339, 8064) (15163, 8064) (371, 8064) (5763, 8064) (9400, 8064)


## Model

In [7]:

input_ = Input(shape=(8064,))

bn = BatchNormalization()(input_)
dense = Dense(4096,activation='selu',kernel_initializer='lecun_normal')(bn)
dropout = Dropout(0.05)(dense)

# con = Concatenate()([dropout,input_])
con = Concatenate()([input_,dropout])
bn = BatchNormalization()(con)
dense = Dense(4096,activation='selu',kernel_initializer='lecun_normal')(bn)
dropout = Dropout(0.05)(dense)


con = Concatenate()([input_,dropout])
# con = Concatenate()([dropout,input_])
# con = K.concatenate([input_,dense])
bn = BatchNormalization()(con)
dense = Dense(4096,activation='selu',kernel_initializer='lecun_normal',kernel_regularizer=l2(0.000004))(bn)
dropout = Dropout(0.055)(dense)

con = Concatenate()([input_,dropout])
# con = Concatenate()([dropout,input_])
bn = BatchNormalization()(con)
dense = Dense(4096,activation='selu',kernel_initializer='lecun_normal',kernel_regularizer=l2(0.000004))(bn)
dropout = Dropout(0.05)(dense)

con = Concatenate()([input_,dropout])
# con = Concatenate()([dropout,input_])
# con = K.concatenate([input_,dense])
bn = BatchNormalization()(con)
dense = Dense(2048,activation='selu',kernel_initializer='lecun_normal',kernel_regularizer=l2(0.000002))(bn)
dropout = Dropout(0.06)(dense)

con = Concatenate()([input_,dropout])
# con = Concatenate()([dropout,input_])
bn = BatchNormalization()(con)
dense = Dense(2048,activation='selu',kernel_initializer='lecun_normal',kernel_regularizer=l2(0.000002))(bn)
dropout = Dropout(0.06)(dense)

# # con = Concatenate()([dropout,input_])
# con = Concatenate()([input_,dropout])
# # con = K.concatenate([input_,dense])
# bn = BatchNormalization()(con)
# dense = Dense(1024,activation='selu',kernel_initializer='lecun_normal',kernel_regularizer=l2(0.000001))(bn)
# dropout = Dropout(0.065)(dense)

# con = Concatenate()([input_,dropout])
# # con = Concatenate()([dropout,input_])
# bn = BatchNormalization()(con)
# dense = Dense(1024,activation='selu',kernel_initializer='lecun_normal',kernel_regularizer=l2(0.000001))(bn)
# dropout = Dropout(0.065)(dense)

# # con = Concatenate()([dropout,input_])
# # con = Concatenate()([dropout,input_])
# con = Concatenate()([input_,dropout])
# bn = BatchNormalization()(con)
# dense = Dense(512,activation='selu',kernel_initializer='lecun_normal')(bn)
# dropout = Dropout(0.07)(dense)

# # con = Concatenate()([dropout,input_])
# con = Concatenate()([input_,dropout])
# bn = BatchNormalization()(con)
# dense = Dense(512,activation='selu',kernel_initializer='lecun_normal')(bn)
# dropout = Dropout(0.07)(dense)

# # con = Concatenate()([dropout,input_])
# # con = Concatenate()([dropout,input_])
# # con = K.concatenate([input_,dense])
# bn = BatchNormalization()(dropout)
# dense = Dense(64,activation='selu',kernel_initializer='lecun_normal')(bn)
# dropout = Dropout(0.2)(dense)

# # con = Concatenate()([dropout,input_])
# # con = Concatenate()([dropout,input_])
# bn = BatchNormalization()(dropout)
# dense = Dense(64,activation='selu',kernel_initializer='lecun_normal')(bn)
# dropout = Dropout(0.2)(dense)

# con = Concatenate()([input_,dropout])
# con = Concatenate()([dropout,input_])
bn = BatchNormalization()(dropout)
dense = Dense(41,activation='softmax',kernel_initializer='lecun_normal')(bn)
# dense = Dense(41,activation='softmax',kernel_initializer='uniform',kernel_regularizer=l2(0.000008))(dropout)
# dense = LeakyReLU()(dense)
model = Model(inputs=input_, outputs=dense)
# model = Sequential()
# i
# model.add(BatchNormalization())
# model.add(Dense(41,activation='linear',input_shape=(41,)))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 8064)         0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 8064)         32256       input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 4096)         33034240    batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 4096)         0           dense_1[0][0]                    
__________________________________________________________________________________________________
concatenat

In [9]:
X_train = np.concatenate((X_train,X_semi))

In [11]:
Y_train = np.concatenate((Y_train,Y_semi))

In [12]:
reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.5, patience=10, min_lr=1e-4)
# ton = TerminateOnNaN()

model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
model.fit(X_train,Y_train,epochs=10000,validation_data=(X_valid,Y_valid),batch_size=64)#callbacks=[reduce_lr])

Train on 18502 samples, validate on 371 samples
Epoch 1/10000
18502/18502 [==============================] - 40s 2ms/step - loss: 3.4166 - acc: 0.1304 - val_loss: 5.9909 - val_acc: 0.0809
Epoch 2/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 3.2276 - acc: 0.1842 - val_loss: 4.3838 - val_acc: 0.0216
Epoch 3/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 3.1274 - acc: 0.2023 - val_loss: 4.4900 - val_acc: 0.0485
Epoch 4/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 3.0394 - acc: 0.2258 - val_loss: 5.9567 - val_acc: 0.0485
Epoch 5/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 2.9844 - acc: 0.2429 - val_loss: 4.3588 - val_acc: 0.0377
Epoch 6/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 2.9310 - acc: 0.2570 - val_loss: 4.8975 - val_acc: 0.0162
Epoch 7/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 2.9341 - acc: 0.2638 - val_los

Epoch 59/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 2.1500 - acc: 0.4416 - val_loss: 4.2910 - val_acc: 0.0485
Epoch 60/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 2.1470 - acc: 0.4369 - val_loss: 3.9845 - val_acc: 0.0485
Epoch 61/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 2.1302 - acc: 0.4452 - val_loss: 4.0324 - val_acc: 0.0485
Epoch 62/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 2.1227 - acc: 0.4435 - val_loss: 4.2259 - val_acc: 0.0296
Epoch 63/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 2.1267 - acc: 0.4450 - val_loss: 4.0906 - val_acc: 0.0350
Epoch 64/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 2.1219 - acc: 0.4492 - val_loss: 4.3099 - val_acc: 0.0485
Epoch 65/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 2.1209 - acc: 0.4466 - val_loss: 4.2482 - val_acc: 0.0296
Epoch 66/1000

Epoch 117/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 1.8766 - acc: 0.5114 - val_loss: 4.0044 - val_acc: 0.0270
Epoch 118/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 1.8702 - acc: 0.5095 - val_loss: 3.9760 - val_acc: 0.0485
Epoch 119/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 1.8875 - acc: 0.5092 - val_loss: 4.1208 - val_acc: 0.0162
Epoch 120/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 1.8754 - acc: 0.5111 - val_loss: 3.9506 - val_acc: 0.0485
Epoch 121/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 1.8740 - acc: 0.5098 - val_loss: 3.9304 - val_acc: 0.0485
Epoch 122/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 1.8524 - acc: 0.5150 - val_loss: 4.0556 - val_acc: 0.0485
Epoch 123/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 1.8592 - acc: 0.5117 - val_loss: 3.9874 - val_acc: 0.0485
Epoch 

Epoch 232/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 1.7099 - acc: 0.5596 - val_loss: 3.8367 - val_acc: 0.0485
Epoch 233/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 1.6991 - acc: 0.5597 - val_loss: 3.8246 - val_acc: 0.0270
Epoch 234/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 1.6915 - acc: 0.5636 - val_loss: 3.8238 - val_acc: 0.0485
Epoch 235/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 1.6857 - acc: 0.5675 - val_loss: 3.9840 - val_acc: 0.0485
Epoch 236/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 1.6932 - acc: 0.5645 - val_loss: 4.0187 - val_acc: 0.0189
Epoch 237/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 1.6956 - acc: 0.5620 - val_loss: 3.8635 - val_acc: 0.0485
Epoch 238/10000
18502/18502 [==============================] - 34s 2ms/step - loss: 1.7000 - acc: 0.5637 - val_loss: 3.8808 - val_acc: 0.0809
Epoch 

ResourceExhaustedError: OOM when allocating tensor with shape[12160,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_2/Adam/gradients/dense_5/weight_regularizer/Square_grad/Mul_1 = Mul[T=DT_FLOAT, _class=["loc:@training_2/Adam/gradients/AddN_16"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_5/kernel/read, training_2/Adam/gradients/dense_5/weight_regularizer/Square_grad/Mul)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: loss_3/add_3/_1125 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_4152_loss_3/add_3", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when 

In [ ]:
for _ in range(num_epochs):
    x, y = generate_data()
    history = model.fit(x, y, epochs=1, batch_size=64)